In [1]:
# 필수 라이브러리 설치
!pip install --quiet ultralytics transformers torch torchvision scikit-learn pillow
!pip install --quiet ftfy regex tqdm git+https://github.com/openai/CLIP.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.6 MB/s eta 0:00:00


In [2]:
# 1) Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, shutil
from concurrent.futures import ThreadPoolExecutor
from ultralytics import YOLO
import zipfile

Mounted at /content/drive
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# ─── 경로 설정 ───────────────────────────────────────────────────────
zip_path    = '/content/drive/MyDrive/prometheus/Archive(2).zip'
extract_dir = '/content/data_extracted'
dataset_dir = '/content/dataset'
train_img   = os.path.join(dataset_dir, 'images/train')
train_lbl   = os.path.join(dataset_dir, 'labels/train')
test_img    = os.path.join(dataset_dir, 'images/test')

In [4]:
# ─── 기존 데이터 제거 후 디렉토리 재생성 ──────────────────────────────
shutil.rmtree(dataset_dir, ignore_errors=True)
shutil.rmtree(extract_dir, ignore_errors=True)
os.makedirs(train_img, exist_ok=True)
os.makedirs(train_lbl, exist_ok=True)
os.makedirs(test_img, exist_ok=True)

In [5]:
# 3) ZIP 압축 해제
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_dir)


In [6]:
# ─── 1) 클래스 루트 자동 감지 ─────────────────────────────────────────
subdirs = [
    d for d in os.listdir(extract_dir)
    if os.path.isdir(os.path.join(extract_dir, d))
]
if len(subdirs) == 1:
    class_root = os.path.join(extract_dir, subdirs[0])
else:
    class_root = extract_dir

# ─── 2) 실제 클래스 디렉터리 로드 ────────────────────────────────────
class_dirs = sorted([
    d for d in os.listdir(class_root)
    if os.path.isdir(os.path.join(class_root, d))
])
print("클래스 개수:", len(class_dirs))


클래스 개수: 98


In [14]:
def find_deepest_folder(folder):
    while True:
        subdirs = [
            d for d in os.listdir(folder)
            if os.path.isdir(os.path.join(folder, d)) and not d.startswith('.')
        ]
        if len(subdirs) == 1:
            folder = os.path.join(folder, subdirs[0])
        else:
            break
    return folder

# 그 다음 라벨 개수 분석
from collections import defaultdict

label_count = defaultdict(int)

for cls in class_dirs:
    cls_folder = find_deepest_folder(os.path.join(class_root, cls))
    files = os.listdir(cls_folder)
    txts = [f for f in files if f.endswith('.txt')]
    label_count[cls] = len(txts)

# 출력
print("⚠️ 클래스별 라벨 수 요약:")
for cls, count in sorted(label_count.items(), key=lambda x: x[1]):
    print(f"{cls}: {count}개")

total = sum(label_count.values())
avg = total / len(label_count)
print(f"\n총 라벨 수: {total}개, 클래스당 평균: {avg:.2f}개")


⚠️ 클래스별 라벨 수 요약:
Sweet rice drink: 100개
Pollack Jjigae: 101개
Pumpkin pancake: 101개
Seasoned chicken: 101개
Spicy fermented skate salad: 101개
Young radish kimchi: 101개
Korean Meat Patties: 102개
Pork back-bone stew: 102개
Potato pancake: 102개
Pumpkin porridge: 102개
Rice cake-Dumpling soup: 102개
Rice skewers: 102개
Scallion Pancake: 102개
Seasoned bean sprouts: 102개
Seasoned cockle clams: 102개
Seasoned soybean sprouts: 102개
Short rib soup: 102개
Shredded daikon: 102개
Sliced boiled meat: 102개
Sliced radish kimchi: 102개
Soft tofu stew: 102개
Soy Sauce Marinated Crab: 102개
Soy braised soybeans: 102개
Soybean noodle soup: 102개
Soybean paste soup: 102개
Soybean paste stew: 102개
Spicy Braised Chicken: 102개
Spicy Braised Seafood: 102개
Spicy Marinated Crab: 102개
Spicy chicken soup: 102개
Spicy fish stew: 102개
Spicy raw fish salad: 102개
Spicy rice cake: 102개
Spicy seafood noodle soup: 102개
Spicy stir fried pork: 102개
Steamed eggs: 102개
Stir fried anchovies: 102개
Stir fried eggplant: 102개
Stir fried fish ca

In [15]:
# 클래스 디렉토리 내 실제 이미지 파일이 들어있는 폴더 추적
def find_deepest_folder(folder):
    # 예: Abalone porridge → Abalone porridge/Abalone porridge → jpg 파일 존재
    while True:
        sub = os.listdir(folder)
        subdirs = [f for f in sub if os.path.isdir(os.path.join(folder, f)) and not f.startswith('.')]
        if len(subdirs) == 1:
            folder = os.path.join(folder, subdirs[0])
        else:
            break
    return folder

# ─── 클래스별 이미지/레이블 정리 (100장만 학습용) ─────────────────────
with ThreadPoolExecutor(max_workers=8) as ex:
    for cls in class_dirs:
        raw_folder = os.path.join(class_root, cls)
        cls_folder = find_deepest_folder(raw_folder)  # 이중 폴더 처리

        files = sorted(os.listdir(cls_folder))

        # 모든 .jpg와 .txt 쌍 수집
        paired = [(f, f[:-4] + '.txt') for f in files if f.endswith('.jpg') and f[:-4] + '.txt' in files]

        learned = set()
        for img, txt in paired:  # ✅ 변경된 부분: 모든 레이블 이미지 사용
            learned.add(img)
            src_img = os.path.join(cls_folder, img)
            src_lbl = os.path.join(cls_folder, txt)
            dst_img = os.path.join(train_img, f'{cls}_{img}')
            dst_lbl = os.path.join(train_lbl, f'{cls}_{txt}')
            ex.submit(shutil.copy, src_img, dst_img)
            ex.submit(shutil.copy, src_lbl, dst_lbl)

        # 나머지 .jpg → 테스트 이미지로 복사
        for f in files:
            if f.endswith('.jpg') and f not in learned:
                src = os.path.join(cls_folder, f)
                dst = os.path.join(test_img, f'{cls}_{f}')
                ex.submit(shutil.copy, src, dst)

# ─── 개수 확인 ────────────────────────────────────────────────────────
print("▶ 학습용 이미지 수:", len(os.listdir(train_img)))
print("▶ 학습용 라벨 수 :", len(os.listdir(train_lbl)))
print("▶ 테스트 이미지 수:", len(os.listdir(test_img)))

▶ 학습용 이미지 수: 9265
▶ 학습용 라벨 수 : 9265
▶ 테스트 이미지 수: 18674


In [ ]:
# dataset.yaml 생성 (YOLOv8 학습 설정)
yaml_path = '/content/dataset.yaml'
with open(yaml_path, 'w') as f:
    f.write(f"""
path: /content/dataset
train: images/train
val: images/train
names: [{", ".join(f'"{cls}"' for cls in class_dirs)}]
""")

print(f"✅ dataset.yaml 생성 완료 → {yaml_path}")


✅ dataset.yaml 생성 완료 → /content/dataset.yaml


In [ ]:
!ls /content | grep dataset.yaml


dataset.yaml


In [ ]:
from ultralytics import YOLO
import os

# 모델 초기화 (사전학습된 yolov8s 사용)
model = YOLO('yolov8s.pt')

# 클래스 이름 등록 (dataset.yaml과 일치)
model.model.names = class_dirs

# 경로 설정
drive_root = '/content/drive/MyDrive/prometheus'

# 모델 훈련
results = model.train(
    data='/content/dataset.yaml',         # ✅ yaml 경로 정확히 지정
    epochs=100,
    imgsz=640,
    batch=32,
    lr0=0.001,
    lrf=0.2,
    cos_lr=True,
    augment=True,
    mosaic=1.0,
    mixup=0.2,
    copy_paste=0.3,
    auto_augment='albumentations',
    patience=30,
    val=True,
    split=0.15,                           # 학습 이미지 중 15%는 validation
    project=os.path.join(drive_root, 'models'),
    name='yolov8_all_labeled'             # ✅ 이름도 "100 labels" → "all labeled"로 더 정확하게 변경
)

# 완료 메시지 출력
print("✅ Training complete. Weights saved to:")
print(os.path.join(drive_root, 'models/yolov8_all_labeled/weights/best.pt'))


Ultralytics 8.3.155 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=albumentations, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.3, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.2, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_all_labeled2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=30, perspective=0.0, plots=True, pose=12.0, pr

100%|██████████| 755k/755k [00:00<00:00, 93.9MB/s]

Overriding model.yaml nc=80 with nc=98

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

 22        [15, 18, 21]  1   2153974  ultralytics.nn.modules.head.Detect           [98, [128, 256, 512]]         
Model summary: 129 layers, 11,173,526 parameters, 11,173,510 gradients, 28.9 GFLOPs

Transferred 349/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 344MB/s]


...

     83/100      7.14G     0.7299     0.7941      1.175         48        640: 100%|██████████| 289/289 [03:55<00:00,  1.23it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 145/145 [01:44<00:00,  1.39it/s]


                   all       9232       9432      0.986      0.985      0.993      0.885

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     84/100      7.14G     0.7299     0.7931      1.177         59        640: 100%|██████████| 289/289 [03:58<00:00,  1.21it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 145/145 [01:43<00:00,  1.41it/s]


                   all       9232       9432      0.985      0.986      0.993      0.887

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     85/100      7.14G     0.7325     0.7763      1.176        106        640:  54%|█████▍    | 157/289 [02:07<01:50,  1.19it/s]

In [ ]:
import os

base_dir = '/content/drive/MyDrive/prometheus/models'
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith('.csv') or file.endswith('.pt'):
            print(os.path.join(root, file))


/content/drive/MyDrive/prometheus/models/myexp/results.csv
/content/drive/MyDrive/prometheus/models/myexp/weights/last.pt
/content/drive/MyDrive/prometheus/models/myexp/weights/best.pt
/content/drive/MyDrive/prometheus/models/myexp2/results.csv
/content/drive/MyDrive/prometheus/models/myexp2/weights/last.pt
/content/drive/MyDrive/prometheus/models/myexp2/weights/best.pt
/content/drive/MyDrive/prometheus/models/yolov8_all_labeled2/results.csv
/content/drive/MyDrive/prometheus/models/yolov8_all_labeled2/weights/last.pt
/content/drive/MyDrive/prometheus/models/yolov8_all_labeled2/weights/best.pt


In [ ]:
import pandas as pd

results_path = '/content/drive/MyDrive/prometheus/models/yolov8_all_labeled2/results.csv'
df = pd.read_csv(results_path)

print("📌 실제 컬럼 목록:")
print(df.columns.tolist())


📌 실제 컬럼 목록:
['epoch', 'time', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss', 'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss', 'lr/pg0', 'lr/pg1', 'lr/pg2']


In [ ]:
import pandas as pd

# 파일 경로
results_path = '/content/drive/MyDrive/prometheus/models/yolov8_all_labeled2/results.csv'
df = pd.read_csv(results_path)

# 공백 제거 (안해도 되지만 안전하게)
df.columns = df.columns.str.strip()

# best mAP50 기준 에폭 확인
best_epoch = df['metrics/mAP50(B)'].idxmax()
best_map50 = df['metrics/mAP50(B)'].max()

print(f'✅ best.pt는 {best_epoch} 에폭에서 저장됨 (mAP50: {best_map50:.3f})')


✅ best.pt는 83 에폭에서 저장됨 (mAP50: 0.993)


In [ ]:
from ultralytics import YOLO

# best.pt에서 시작
model = YOLO('/content/drive/MyDrive/prometheus/models/yolov8_all_labeled2/weights/best.pt')

# 추가로 17 에폭 학습
model.train(
    data='/content/dataset.yaml',
    epochs=17,
    project='/content/drive/MyDrive/prometheus/models',
    name='yolov8_all_labeled2_resume',
    pretrained=True  # best.pt를 기반으로 학습
)


Ultralytics 8.3.155 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=17, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/prometheus/models/yolov8_all_labeled2/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_all_labeled2_resume, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, ove

100%|██████████| 755k/755k [00:00<00:00, 24.1MB/s]


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

 22        [15, 18, 21]  1   2153974  ultralytics.nn.modules.head.Detect           [98, [128, 256, 512]]         
Model summary: 129 layers, 11,173,526 parameters, 11,173,510 gradients, 28.9 GFLOPs

Transferred 355/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 113MB/s]


...

      17/17      6.48G      0.469     0.2913      1.051         16        640: 100%|██████████| 577/577 [03:18<00:00,  2.91it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 289/289 [01:34<00:00,  3.05it/s]


                   all       9232       9432      0.994      0.991      0.994      0.908

17 epochs completed in 1.412 hours.
Optimizer stripped from /content/drive/MyDrive/prometheus/models/yolov8_all_labeled2_resume/weights/last.pt, 22.6MB
Optimizer stripped from /content/drive/MyDrive/prometheus/models/yolov8_all_labeled2_resume/weights/best.pt, 22.6MB

Validating /content/drive/MyDrive/prometheus/models/yolov8_all_labeled2_resume/weights/best.pt...
Ultralytics 8.3.155 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,163,510 parameters, 0 gradients, 28.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 289/289 [01:38<00:00,  2.95it/s]


                   all       9232       9432      0.993      0.991      0.994      0.908
      Abalone porridge        103        107          1      0.994      0.995      0.891
     Acorn jelly salad        101        102      0.998       0.99      0.995      0.903
     Bean sprouts soup        101        101          1      0.977      0.995      0.896
    Beef tripe hot pot        105        105      0.999          1      0.995      0.891
    Black Bean Noodles         99         99          1          1      0.995        0.9
      Boiled pork wrap        103        105      0.981      0.964      0.993      0.849
   Braised Cutlassfish        102        110      0.983      0.964      0.994      0.888
      Braised Mackerel        173        177          1      0.992      0.995        0.9
         Braised Saury         92         93          1      0.996      0.995      0.883
          Braised Tofu        102        103       0.99      0.952      0.994      0.907
        Braised kimch

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a53d5d50d90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019

In [7]:
import os

# 확인할 경로
output_path = '/content/drive/MyDrive/prometheus/models/yolov8_all_labeled2_resume/weights'

# 결과 파일 리스트 출력
if os.path.exists(output_path):
    print("저장된 파일 목록:", os.listdir(output_path))
else:
    print("해당 경로가 존재하지 않습니다.")


저장된 파일 목록: ['best.pt', 'last.pt']


In [8]:
# 한국어 클래스명 리스트 (순서 유지 필수)
korean_names = [
    '전복죽', '도토리묵 무침', '콩나물국', '소곱창전골', '짜장면', '수육 쌈', '갈치조림', '고등어조림', '꽁치조림', '두부조림',
    '김치찜', '감자조림', '갈비찜', '메밀국수', '배추김치', '쫄면', '냉면', '만두', '계란국', '잔치국수',
    '생선전', '치킨', '계란후라이', '고추튀김', '새우튀김', '오징어튀김', '삼계탕', '수제비', '약과', '약밥',
    '육개장', '김치찌개', '김치전', '칼국수', '동그랑땡', '닭볶음탕', '반달떡', '계란말이', '약밥', '한과',
    '추어탕', '비빔냉면', '설렁탕', '깻잎장아찌', '족발', '동태찌개', '감자탕', '감자전', '호박전', '호박죽',
    '떡만두국', '떡꼬치', '파전', '콩나물무침', '닭무침', '꼬막무침', '고사리나물', '숙주나물', '갈비탕', '무생채',
    '편육', '깍두기', '순두부찌개', '간장게장', '콩자반', '콩국수', '된장국', '된장찌개', '닭도리탕', '해물찜',
    '양념게장', '닭개장', '홍어무침', '매운탕', '회무침', '떡볶이', '짬뽕', '제육볶음', '계란찜', '멸치볶음',
    '가지볶음', '어묵볶음', '잡채', '소시지볶음', '미역줄기볶음', '오징어채볶음', '진미채볶음', '애호박볶음', '꽈리고추볶음', '감자채볶음',
    '건새우볶음', '순대', '경단', '식혜', '두부김치', '백김치', '열무김치', '열무국수'
]


In [11]:
# dataset.yaml 생성
yaml_path = '/content/dataset.yaml'
with open(yaml_path, 'w') as f:
    f.write(f"""
path: /content/dataset
train: images/train
val: images/train
names: [{", ".join(f'"{cls}"' for cls in korean_names)}]
""")

print(f"✅ 한국어 dataset.yaml 생성 완료 → {yaml_path}")

✅ 한국어 dataset.yaml 생성 완료 → /content/dataset.yaml


In [16]:
from ultralytics import YOLO

# 기존 영어로 학습된 best.pt 경로
pretrained_path = '/content/drive/MyDrive/prometheus/models/yolov8_all_labeled2_resume/weights/best.pt'

# 모델 로드
model = YOLO(pretrained_path)

# 추가 학습 (한글 클래스명 포함)
model.train(
    data=yaml_path,
    epochs=10,
    project='/content/drive/MyDrive/prometheus/models',
    name='yolov8_all_labeled2_resume',
    pretrained=True
)

Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/prometheus/models/yolov8_all_labeled2_resume/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_all_labeled2_resume3, nbs=64, nms=False, opset=None, optimize=False, optimizer=a

train: Scanning /content/dataset/labels/train... 9232 images, 31 backgrounds, 33 corrupt: 100%|██████████| 9265/9265 [00:16<00:00, 548.34it/s] 

train: /content/dataset/images/train/Acorn jelly salad_Img_064_0229.jpg: corrupt JPEG restored and saved
train: /content/dataset/images/train/Acorn jelly salad_Img_064_0254.jpg: corrupt JPEG restored and saved
train: /content/dataset/images/train/Acorn jelly salad_Img_064_0256.jpg: corrupt JPEG restored and saved
train: /content/dataset/images/train/Acorn jelly salad_Img_064_0264.jpg: corrupt JPEG restored and saved
train: /content/dataset/images/train/Acorn jelly salad_Img_064_0283.jpg: corrupt JPEG restored and saved
train: /content/dataset/images/train/Bean sprouts soup_Img_021_0026.jpg: corrupt JPEG restored and saved
train: /content/dataset/images/train/Bean sprouts soup_Img_021_0255.jpg: corrupt JPEG restored and saved
train: /content/dataset/images/train/Bean sprouts soup_Img_021_0269.jpg: corrupt JPEG restored and saved
train: /content/dataset/images/train/Black Bean Noodles_Img_057_0021.jpg: corrupt JPEG restored and saved
train: /content/dataset/images/train/Black Bean Noodle

train: New cache created: /content/dataset/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 736.8±741.6 MB/s, size: 86.4 KB)


val: Scanning /content/dataset/labels/train.cache... 9232 images, 31 backgrounds, 33 corrupt: 100%|██████████| 9265/9265 [00:00<?, ?it/s]

...

      10/10      6.75G     0.4424     0.2718      1.025         16        640: 100%|██████████| 577/577 [03:23<00:00,  2.84it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 289/289 [01:33<00:00,  3.09it/s]


                   all       9232       9432      0.994      0.994      0.994      0.919

10 epochs completed in 0.833 hours.
Optimizer stripped from /content/drive/MyDrive/prometheus/models/yolov8_all_labeled2_resume3/weights/last.pt, 22.6MB
Optimizer stripped from /content/drive/MyDrive/prometheus/models/yolov8_all_labeled2_resume3/weights/best.pt, 22.6MB

Validating /content/drive/MyDrive/prometheus/models/yolov8_all_labeled2_resume3/weights/best.pt...
Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,163,510 parameters, 0 gradients, 28.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 289/289 [01:38<00:00,  2.93it/s]


                   all       9232       9432      0.995      0.994      0.994      0.919
                   전복죽        103        107          1      0.995      0.995      0.907
               도토리묵 무침        101        102      0.999       0.99      0.995       0.91
                  콩나물국        101        101       0.99      0.983      0.995      0.904
                 소곱창전골        105        105      0.998          1      0.995      0.906
                   짜장면         99         99      0.998          1      0.995      0.916
                  수육 쌈        103        105      0.999      0.981      0.995      0.892
                  갈치조림        102        110      0.991       0.99      0.995      0.915
                 고등어조림        173        177      0.999      0.994      0.995      0.913
                  꽁치조림         92         93          1      0.994      0.995      0.895
                  두부조림        102        103       0.99       0.99      0.995      0.918
                   김치

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc4860b5d10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019

In [18]:
from ultralytics import YOLO

# 학습이 끝난 최신 best.pt 경로
model_path = '/content/drive/MyDrive/prometheus/models/yolov8_all_labeled2_resume3/weights/best.pt'

# 모델 로드
model = YOLO(model_path)

# 클래스명 출력
print("✅ 모델에 저장된 클래스명:")
print(model.names)


✅ 모델에 저장된 클래스명:
{0: '전복죽', 1: '도토리묵 무침', 2: '콩나물국', 3: '소곱창전골', 4: '짜장면', 5: '수육 쌈', 6: '갈치조림', 7: '고등어조림', 8: '꽁치조림', 9: '두부조림', 10: '김치찜', 11: '감자조림', 12: '갈비찜', 13: '메밀국수', 14: '배추김치', 15: '쫄면', 16: '냉면', 17: '만두', 18: '계란국', 19: '잔치국수', 20: '생선전', 21: '치킨', 22: '계란후라이', 23: '고추튀김', 24: '새우튀김', 25: '오징어튀김', 26: '삼계탕', 27: '수제비', 28: '약과', 29: '약밥', 30: '육개장', 31: '김치찌개', 32: '김치전', 33: '칼국수', 34: '동그랑땡', 35: '닭볶음탕', 36: '반달떡', 37: '계란말이', 38: '약밥', 39: '한과', 40: '추어탕', 41: '비빔냉면', 42: '설렁탕', 43: '깻잎장아찌', 44: '족발', 45: '동태찌개', 46: '감자탕', 47: '감자전', 48: '호박전', 49: '호박죽', 50: '떡만두국', 51: '떡꼬치', 52: '파전', 53: '콩나물무침', 54: '닭무침', 55: '꼬막무침', 56: '고사리나물', 57: '숙주나물', 58: '갈비탕', 59: '무생채', 60: '편육', 61: '깍두기', 62: '순두부찌개', 63: '간장게장', 64: '콩자반', 65: '콩국수', 66: '된장국', 67: '된장찌개', 68: '닭도리탕', 69: '해물찜', 70: '양념게장', 71: '닭개장', 72: '홍어무침', 73: '매운탕', 74: '회무침', 75: '떡볶이', 76: '짬뽕', 77: '제육볶음', 78: '계란찜', 79: '멸치볶음', 80: '가지볶음', 81: '어묵볶음', 82: '잡채', 83: '소시지볶음', 84: '미역줄기볶음', 85: '오징어채볶음', 86: